In [1]:
import pandas as pd
import pickle
import numpy as np
import os

from google.colab import files
import io
import time

In [2]:
# mounting google drive 
from google.colab import drive
drive.mount('/content/drive')

# specify base directory for files
baseDir = '/content/drive/My Drive/ETHZ/AML/Project_2'

Mounted at /content/drive


# Importing data --> data is complete, no imputing needed

In [5]:
# Importing original data & imputing
D_X_df = pd.read_csv(baseDir + 'X_train.csv', index_col = 'id')
D_y_df = pd.read_csv(baseDir + 'y_train.csv', index_col = 'id')
D_test_df = pd.read_csv(baseDir + 'X_test.csv', index_col = 'id')

D_X_df.head(20)

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,...,x960,x961,x962,x963,x964,x965,x966,x967,x968,x969,x970,x971,x972,x973,x974,x975,x976,x977,x978,x979,x980,x981,x982,x983,x984,x985,x986,x987,x988,x989,x990,x991,x992,x993,x994,x995,x996,x997,x998,x999
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,-1.498973,1.448201,2.784979,1.905992,1.285007,-0.571679,1.253798,-2.590709,1.379211,-1.553323,-0.564972,2.584993,0.011699,0.441804,-0.699960,1.747325,2.065368,2.787858,2.013942,-1.401352,2.147372,0.617870,2.141796,-2.781471,0.413109,2.345852,0.030399,0.887061,-1.232804,0.109728,4.319156,3.178303,0.184391,-1.243393,1.406802,3.132227,-0.215465,1.505367,-0.503517,0.270487,...,1.348220,-1.877967,-0.739429,1.566261,2.730320,-2.380653,-2.652265,0.176041,2.008110,0.240972,0.710862,-1.695216,4.116688,0.849540,0.958647,0.514287,-2.362364,0.145864,0.874874,-1.650551,-0.709709,1.894579,2.585351,1.893157,-1.464831,4.246892,-4.139556,-2.965517,3.240977,1.190587,2.638401,-1.365574,2.856497,-1.916006,1.406900,-2.581604,0.839115,-0.004883,3.173465,2.179183
1.0,-0.521470,-0.493049,0.891382,-0.080855,0.227825,-0.167394,-0.426608,0.371071,-0.065361,-0.271039,-0.074448,0.964929,1.471138,0.432700,-1.190865,0.030373,0.597827,0.996612,-0.523518,-0.668377,0.379092,-0.797149,0.194128,-0.873084,0.268315,0.130959,0.547615,0.430073,0.237003,-0.476202,-0.005975,-0.197892,-0.400580,-1.111818,-0.140526,-0.209191,1.419397,-0.518544,1.120603,-0.749909,...,-0.495387,-0.046147,-0.066997,-0.003721,-0.361316,-0.048866,-0.254264,-0.421037,0.487685,0.770651,0.280975,0.158271,-0.809737,0.652606,-0.397294,0.051201,-0.810056,0.412149,0.503858,-0.645113,0.650803,0.799408,-0.596739,-0.804695,-0.803387,0.417222,-0.041625,0.352568,1.182781,-0.090232,0.662354,-0.443464,-0.540985,-0.164082,0.223598,0.274742,-0.122392,0.971394,0.604963,0.355499
2.0,-0.417724,-0.019106,0.938377,-0.670472,0.298922,0.917788,0.189585,-0.259406,0.591056,-1.391407,0.486646,0.704700,0.844097,1.195779,-0.673821,-0.572311,0.652869,0.782490,-0.746676,-0.972831,0.533623,0.977401,0.360857,0.497896,0.353269,0.142392,0.399858,0.820036,-0.326413,0.108279,0.979080,-0.327711,0.796991,-0.678033,-0.149652,0.651686,0.436569,-0.466789,0.181601,0.901009,...,-1.041419,-1.554904,0.393147,-0.085724,0.346211,-0.719380,-1.293079,0.622270,1.113963,1.211985,0.193227,0.007382,0.704057,0.383336,-1.380809,-0.570085,-1.476090,0.026893,0.457055,-1.311419,0.961965,1.047396,0.179421,1.126904,-1.251237,0.331101,-1.254555,-0.535584,-0.015067,-0.035772,0.617464,-0.543036,-0.321695,-1.778676,1.118608,-0.937445,-0.239242,0.842709,2.086818,0.512741
3.0,-0.471972,0.000398,0.784836,1.088817,-0.436292,0.023086,0.611958,-0.720903,0.310497,-0.703081,-0.595316,0.522094,0.075199,0.997927,0.952303,-0.000920,1.353171,0.492778,-0.100945,-1.047399,0.830849,1.096135,0.571503,-1.033216,-0.796911,1.031535,0.838741,-0.000040,-0.757737,-0.538154,2.045717,1.126708,0.239202,-0.506557,-0.981757,1.753852,-0.302296,0.940582,-0.913573,0.615190,...,-0.252032,-1.374432,-0.952914,1.271367,1.055987,-1.535406,-1.647291,0.222529,1.428344,0.926888,0.582608,-0.211813,1.205523,0.536371,-0.335546,-0.458197,-0.087182,0.327218,0.070464,-0.018343,-0.014398,0.195694,1.137555,1.080092,-0.282592,1.686490,-1.047579,-1.154139,0.579924,1.122358,0.672421,-1.942345,0.366181,-1.226904,1.152494,-0.104389,0.702375,0.426496,0.336746,1.304973
4.0,0.201026,-0.579901,0.638809,-0.614121,0.468388,0.535726,0.271890,0.054270,0.297078,-0.677568,0.268948,0.202088,1.004802,0.001226,-0.521941,-0.258955,0.442525,0.543422,0.000783,-0.758873,0.082507,0.595247,-0.023682,0.520904,0.323870,0.591798,-0.032731,-0.618663,-1.044747,-0.418920,0.489303,-0.882979,0.568533,-0.337236,-0.097782,-0.371909,0.520280,-0.358214,-0.096405,-0.518626,...,-0.250465,-0.331456,-0.272372,-1.008573,-0.203979,0.105460,0.146728,-0.068328,0.018873,0.687485,-0.606792,-0.288901,0.066643,0

# Convert to np & normalize to mean=0 , std=1

In [12]:
from sklearn import preprocessing

D_X = np.array(D_X_df)
D_y = np.array(D_y_df)
D_test = np.array(D_test_df)

scaler = preprocessing.StandardScaler()
scaler.fit_transform(D_X)

D_X_stand = scaler.transform(D_X)
D_test_stand = scaler.transform(D_test) # scaling accoring to train set or to test set?

# stratified K-fold

In [22]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=13) # random_state is seed value

ind_splits_skf = list(skf.split(D_X_stand,D_y))

# Feature selection - TODO?

# Fitting data - one vs one classifier

In [36]:
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsOneClassifier

X_fold_fit = D_X_stand[ind_splits_skf[0][0]]
y_fold_fit = D_y[ind_splits_skf[0][0]]

X_fold_pred = D_X_stand[ind_splits_skf[0][1]]
y_fold_pred = D_y[ind_splits_skf[0][1]]

model = OneVsOneClassifier(LinearSVC(random_state=0)).fit(X_fold_fit, y_fold_fit)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


# Prediction & BMAC on Stratified Kfold

In [39]:
from sklearn.metrics import balanced_accuracy_score

BMAC = balanced_accuracy_score(y_fold_pred, model.predict(X_fold_pred))
BMAC

0.5936111111111111

# Predictions on test set

In [41]:
test_pred = model.predict(D_test_stand)

df_pred = pd.DataFrame(test_pred, index=D_test_df.index)

In [49]:
name = f'{baseDir}predictions/pred_{unixTime}.csv'
name

'/content/drive/My Drive/ETHZ/AML/Project_2/predictions/pred_1603589606.csv'

# Saving dataframe

In [50]:
import datetime
now = datetime.datetime.now()
unixTime = round(now.timestamp())

name = f'{baseDir}predictions/pred_{unixTime}.csv'
print(f'File was saved under {name}')

df_pred.to_csv(name, index=True, header = True, float_format='%.3f') #, compression='zip')

File was saved under /content/drive/My Drive/ETHZ/AML/Project_2/predictions/pred_1603589667.csv
